---
title: "Milestone 4"
subtitle: "Chatbot Evaluation"
date: today
author: Jaykumar Patel, Laith Altarabishi, Harold Zhong
format:
  html:
    toc: true
    embed-resources: true
mainfont: TeX Gyre Schola
monofont: JetBrainsMono Nerd Font
mathfont: TeX Gyre Schola Math Regular
jupyter: python3
---


# Intro

This documents the evaluation of our chatbot.

Note: You will also need PyTorch and CUDA installed. We don't include detailed instructions on installing PyTorch because we ran this fine-tuning code using [Thunder Compute's A100 GPU](https://www.thundercompute.com/), and we didn't have to manually install PyTorch (it was already pre-installed).

# Install Packages

```python
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

# for Rouge
! pip install evaluate
! pip install rouge_score

# for BertScore
! pip install transformers
! pip install bert-score
```

# Load Original Base Model

We load the original base model `unsloth/unsloth/Qwen2.5-7B`.

```python
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
```

```python
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
```

# Load Fine-Tuned Model

We saved the fine-tuned model and tokenizer to a local directory called `lora_model`. We load our fine-tuned model from that directory.

```python
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
```

```python
fine_tuned_model, fine_tuned_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(fine_tuned_model) # Enable native 2x faster inference
```

# Testing Data Preparation

We load the test data from `abisee/cnn_dailymail`. We use 1% of this data to make the evaluation process more feasible.

```python
from datasets import load_dataset
test_dataset = load_dataset("abisee/cnn_dailymail", name = "3.0.0", split = "test[:1%]")
```

```python
print(len(test_dataset))
print(test_dataset[0])
```

Console Output:

```
115
{'article': '(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony, said it was a move toward greater justice. "As Palestine formally becomes a State Party to the Rome Statute today, the world is also a step closer to ending a long era of impunity and injustice," he said, according to an ICC news release. "Indeed, today brings us closer to our shared goals of justice and peace." Judge Kuniko Ozaki, a vice president of the ICC, said acceding to the treaty was just the first step for the Palestinians. "As the Rome Statute today enters into force for the State of Palestine, Palestine acquires all the rights as well as responsibilities that come with being a State Party to the Statute. These are substantive commitments, which cannot be taken lightly," she said. Rights group Human Rights Watch welcomed the development. "Governments seeking to penalize Palestine for joining the ICC should immediately end their pressure, and countries that support universal acceptance of the court\'s treaty should speak out to welcome its membership," said Balkees Jarrah, international justice counsel for the group. "What\'s objectionable is the attempts to undermine international justice, not Palestine\'s decision to join a treaty to which over 100 countries around the world are members." In January, when the preliminary ICC examination was opened, Israeli Prime Minister Benjamin Netanyahu described it as an outrage, saying the court was overstepping its boundaries. The United States also said it "strongly" disagreed with the court\'s decision. "As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC," the State Department said in a statement. It urged the warring sides to resolve their differences through direct negotiations. "We will continue to oppose actions against Israel at the ICC as counterproductive to the cause of peace," it said. But the ICC begs to differ with the definition of a state for its purposes and refers to the territories as "Palestine." While a preliminary examination is not a formal investigation, it allows the court to review evidence and determine whether to investigate suspects on both sides. Prosecutor Fatou Bensouda said her office would "conduct its analysis in full independence and impartiality." The war between Israel and Hamas militants in Gaza last summer left more than 2,000 people dead. The inquiry will include alleged war crimes committed since June. The International Criminal Court was set up in 2002 to prosecute genocide, crimes against humanity and war crimes. CNN\'s Vasco Cotovio, Kareem Khadder and Faith Karimi contributed to this report.', 'highlights': 'Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .\nIsrael and the United States opposed the move, which could open the door to war crimes investigations against Israelis .', 'id': 'f001ec5c4704938247d27a44948eebb37ae98d01'}
```

# Add formatter

To ensure that the fine-tuned model behaves as predicted, we want to ensure that the text we pass it matches the same format we used for training. To do this, we will use the same Alpaca Prompt template. However, we will set the `output_text` to be `""` since during inference, the output is not defined.

```python
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(input_text):
    instruction = "Given the news article, determine the key highlights."
    output_text = ""
    text = alpaca_prompt.format(instruction, input_text, output_text)
    return text
```

```python
# Testing the formatter
print(formatting_prompts_func(test_dataset[0]['article']))
```

Console Output:

```
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Given the news article, determine the key highlights.

### Input:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ceremony, said it was a move toward greater justice. "As Palestine formally becomes a State Party to the Rome Statute today, the world is also a step closer to ending a long era of impunity and injustice," he said, according to an ICC news release. "Indeed, today brings us closer to our shared goals of justice and peace." Judge Kuniko Ozaki, a vice president of the ICC, said acceding to the treaty was just the first step for the Palestinians. "As the Rome Statute today enters into force for the State of Palestine, Palestine acquires all the rights as well as responsibilities that come with being a State Party to the Statute. These are substantive commitments, which cannot be taken lightly," she said. Rights group Human Rights Watch welcomed the development. "Governments seeking to penalize Palestine for joining the ICC should immediately end their pressure, and countries that support universal acceptance of the court's treaty should speak out to welcome its membership," said Balkees Jarrah, international justice counsel for the group. "What's objectionable is the attempts to undermine international justice, not Palestine's decision to join a treaty to which over 100 countries around the world are members." In January, when the preliminary ICC examination was opened, Israeli Prime Minister Benjamin Netanyahu described it as an outrage, saying the court was overstepping its boundaries. The United States also said it "strongly" disagreed with the court's decision. "As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC," the State Department said in a statement. It urged the warring sides to resolve their differences through direct negotiations. "We will continue to oppose actions against Israel at the ICC as counterproductive to the cause of peace," it said. But the ICC begs to differ with the definition of a state for its purposes and refers to the territories as "Palestine." While a preliminary examination is not a formal investigation, it allows the court to review evidence and determine whether to investigate suspects on both sides. Prosecutor Fatou Bensouda said her office would "conduct its analysis in full independence and impartiality." The war between Israel and Hamas militants in Gaza last summer left more than 2,000 people dead. The inquiry will include alleged war crimes committed since June. The International Criminal Court was set up in 2002 to prosecute genocide, crimes against humanity and war crimes. CNN's Vasco Cotovio, Kareem Khadder and Faith Karimi contributed to this report.

### Response:
```

# Run Inference

Inference is the process of using a trained model to generate predictions or make decisions based on new, previously unseen data.

## Create `get_highlights` function that runs inference

Here, we create a `get_highlights` function, that takes in an article, formats it using the `formatting_prompts_func`, and then passes the formatted prompt to the fine-tuned model to get a response. The response is then returned.

The `get_highlights` function takes a parameter `fine_tuned`. When `fine_tuned` is set to `True`, then the function uses the fine-tuned model to make the inference (generate highlights). When `fine_tuned` is set to `False`, then the function uses the original base model to make the inference (generate highlights).

The `get_highlights` function also calculates the time it takes to run the inference. This information is useful in inference time analysis.

```python
import time

def get_highlights(article, fine_tuned = False):
  start = time.time()
  if fine_tuned:
    inputs = fine_tuned_tokenizer(
    [
      formatting_prompts_func(article)
    ], return_tensors = "pt").to("cuda")

    outputs = fine_tuned_model.generate(**inputs, max_new_tokens = 128)
    decoded_output = fine_tuned_tokenizer.batch_decode(outputs)

    end = time.time()

    return decoded_output[0].split('### Response:')[-1].replace(fine_tuned_tokenizer.eos_token, ''), end - start

  inputs = tokenizer(
  [
    formatting_prompts_func(article)
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 128)
  decoded_output = tokenizer.batch_decode(outputs)

  end = time.time()

  return decoded_output[0].split('### Response:')[-1], end - start
```

## Test `get_highlights` function

Here we test the `get_highlights` function to ensure that it works correctly.

### Test 1

```python
import random

random_index = random.randint(0, len(test_dataset) - 1)
print(f"Testing index: {random_index}")

print("Article:")
print(test_dataset[random_index]['article'])
print()
print("Highlights")
print(test_dataset[random_index]['highlights'])
print()

print("Original Model Test Highlights")
original_model_test = get_highlights(test_dataset[random_index]['article'], fine_tuned=False)
print(original_model_test[0])
print("Original Model Test Time")
print(original_model_test[1])

print()

print("Fine-tuned Model Test Highlights")
fine_tuned_model_test = get_highlights(test_dataset[random_index]['article'], fine_tuned=True)
print(fine_tuned_model_test[0])
print("Fine-tuned Model Test Time")
print(fine_tuned_model_test[1])
```

Console Output:

```
Testing index: 0
Article:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ceremony, said it was a move toward greater justice. "As Palestine formally becomes a State Party to the Rome Statute today, the world is also a step closer to ending a long era of impunity and injustice," he said, according to an ICC news release. "Indeed, today brings us closer to our shared goals of justice and peace." Judge Kuniko Ozaki, a vice president of the ICC, said acceding to the treaty was just the first step for the Palestinians. "As the Rome Statute today enters into force for the State of Palestine, Palestine acquires all the rights as well as responsibilities that come with being a State Party to the Statute. These are substantive commitments, which cannot be taken lightly," she said. Rights group Human Rights Watch welcomed the development. "Governments seeking to penalize Palestine for joining the ICC should immediately end their pressure, and countries that support universal acceptance of the court's treaty should speak out to welcome its membership," said Balkees Jarrah, international justice counsel for the group. "What's objectionable is the attempts to undermine international justice, not Palestine's decision to join a treaty to which over 100 countries around the world are members." In January, when the preliminary ICC examination was opened, Israeli Prime Minister Benjamin Netanyahu described it as an outrage, saying the court was overstepping its boundaries. The United States also said it "strongly" disagreed with the court's decision. "As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC," the State Department said in a statement. It urged the warring sides to resolve their differences through direct negotiations. "We will continue to oppose actions against Israel at the ICC as counterproductive to the cause of peace," it said. But the ICC begs to differ with the definition of a state for its purposes and refers to the territories as "Palestine." While a preliminary examination is not a formal investigation, it allows the court to review evidence and determine whether to investigate suspects on both sides. Prosecutor Fatou Bensouda said her office would "conduct its analysis in full independence and impartiality." The war between Israel and Hamas militants in Gaza last summer left more than 2,000 people dead. The inquiry will include alleged war crimes committed since June. The International Criminal Court was set up in 2002 to prosecute genocide, crimes against humanity and war crimes. CNN's Vasco Cotovio, Kareem Khadder and Faith Karimi contributed to this report.

Highlights
Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .

Original Model Test Highlights

The key highlights of the news article are:

1. The Palestinian Authority officially became the 123rd member of the International Criminal Court (ICC) on Wednesday, giving the court jurisdiction over alleged crimes in Palestinian territories.
2. The Palestinians signed the ICC's founding Rome Statute in January, accepting its jurisdiction over alleged crimes committed in the occupied Palestinian territory, including East Jerusalem, since June 13, 2. The ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis.
 3. As members of the court, Palestinians may be subject to counter-charges as
Original Model Test Time
10.787961959838867

Fine-tuned Model Test Highlights

Palestinian Authority officially becomes 123rd member of the International Criminal Court .
Palestinians may be subject to counter-charges as well .
Israel and the United States, neither of which is an ICC member, opposed the move .
Fine-tuned Model Test Time
5.0052711963653564
```

### Test 2

Here we test the `get_highlights` function again to ensure that it works correctly.

```python
random_index_2 = random.randint(0, len(test_dataset) - 1)
print(f"Testing index: {random_index_2}")

print("Article:")
print(test_dataset[random_index_2]['article'])
print()
print("Highlights")
print(test_dataset[random_index_2]['highlights'])
print()

print("Original Model Test Highlights")
original_model_test_2 = get_highlights(test_dataset[random_index_2]['article'], fine_tuned=False)
print(original_model_test_2[0])
print("Original Model Test Time")
print(original_model_test_2[1])

print()

print("Fine-tuned Model Test Highlights")
fine_tuned_model_test_2 = get_highlights(test_dataset[random_index_2]['article'], fine_tuned=True)
print(fine_tuned_model_test_2[0])
print("Fine-tuned Model Test Time")
print(fine_tuned_model_test_2[1])
```

Console Output:

```
Testing index: 81
Article:
(CNN)One hundred and forty-seven victims. Many more families affected. Even more broken hopes and dreams. As Kenyans mourned those killed last week in one of the deadliest terrorist attacks in the nation, citizens used social media to share the victims' stories, hopes and dreams. Using the hashtag #147notjustanumber -- a reference to the number of people, mostly students, killed at Garissa University College on Thursday -- Kenyans tweeted pictures of the victims in happier times. Kenyan authorities have not released a list of the victims. The posts provided heart-wrenching details on the victims, including one about an elderly man whose dreams died with his son. He had reportedly taken a loan to educate him at the university, where he was killed by Al-Shabaab terrorists. The attack in Kenya killed 142 students, three security officers and two university security personnel, and was the nation's deadliest since the bombing of the U.S. Embassy in 1998. Kenyan churches mourned the dead during Easter services Sunday as armed guards protected the congregations. In emotional services nationwide, churchgoers wept as they paid tribute to the victims of the massacre. The gunmen who attacked the university in the predawn hours separated Muslims from Christians and killed the latter. The extremist group has also killed Muslims in recent attacks. The Interior Ministry has identified one of the attackers killed by security forces as the son of a government official. The father of suspect Abdirahim Abdullahi is a chief in Mandera and had reported his son missing, officials said. The Islamist extremist group is based in Somalia, but it hasn't confined its terrorism to the nation that shares a border with Kenya. In 2013, militants attacked Nairobi's upscale Westgate Mall, killing nearly 70 people.

Highlights
Kenyans use hashtag #147notjustanumber to honor victims of Kenya university attack .
The attack killed 142 students, three security officers and two university security personnel .

Original Model Test Highlights

The key highlights of the news article are:

1. **Massive Attack**: A terrorist attack occurred at Garissa University College, resulting in 147 deaths, making it one of the deadliest in Kenya.
2. **Victims' Stories**: Kenyans shared the victims' stories, hopes, and dreams on social media using the hashtag #147notjustanumber.
3. **Attack Details**: The attackers separated Muslims from Christians and targeted Christians, while the group has also killed Muslims in recent attacks.
4. **Suspect Information**: One of the attackers was identified as the son of a government official, whose
Original Model Test Time
9.827091217041016

Fine-tuned Model Test Highlights

Kenyans share stories of victims on social media using #147notjustanumber .
The attack in Kenya killed 142 students, three security officers and two university personnel .
The gunmen who attacked the university in the predawn hours separated Muslims from Christians .
Fine-tuned Model Test Time
6.604422569274902
```

## Run Inference on Testing Data

```python
import json
from tqdm import tqdm

final_data = []
for test_item in tqdm(test_dataset):
  ground_truth_highlights = test_item['highlights']
  original_model_highlights, original_model_time = get_highlights(test_item['article'], fine_tuned = False)
  fine_tuned_model_highlights, fine_tuned_model_time = get_highlights(test_item['article'], fine_tuned = True)

  final_data.append({
    "article": test_item['article'],
    "ground_truth_all_highlights": ground_truth_highlights,
    "original_model_all_data": (original_model_highlights, original_model_time),
    "fine_tuned_model_all_data": (fine_tuned_model_highlights, fine_tuned_model_time)
  })

ground_truth_all_highlights = [data['ground_truth_all_highlights'] for data in final_data]

original_model_all_highlights = [data['original_model_all_data'][0] for data in final_data]
original_model_all_times = [data['original_model_all_data'][1] for data in final_data]

fine_tuned_model_all_highlights = [data['fine_tuned_model_all_data'][0] for data in final_data]
fine_tuned_model_all_times = [data['fine_tuned_model_all_data'][1] for data in final_data]

# save data to file for backup
with open("final_data.json", "w") as f:
  json.dump(final_data, f)
```

# Evaluate

## Rogue

Here, we use the `evaluate` library to calculate the Rogue Score for the highlights generated by the fine-tuned model and the original base model. We use the highlights in the original dataset as reference.

We followed this [tutorial](https://medium.com/@eren9677/text-summarization-387836c9e178) to write this code.

```python
import evaluate
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions = original_model_all_highlights,
    references = ground_truth_all_highlights
)

fine_tuned_model_results = rouge.compute(
    predictions = fine_tuned_model_all_highlights,
    references = ground_truth_all_highlights
)

print(original_model_results)
print(fine_tuned_model_results)
```

Console Output:

```
{'rouge1': np.float64(0.25593852325683963), 'rouge2': np.float64(0.09879602742572864), 'rougeL': np.float64(0.1845369489659137), 'rougeLsum': np.float64(0.23482225329274253)}
{'rouge1': np.float64(0.3202412916174494), 'rouge2': np.float64(0.1291838902741764), 'rougeL': np.float64(0.23471503036165922), 'rougeLsum': np.float64(0.29682001157527327)}
```

Table View:

| Model | rouge1 | rouge2 | rougeL | rougeLsum |
|-------|--------|--------|--------|-----------|
| Original Base Model | 0.25593852325683963 | 0.09879602742572864 | 0.1845369489659137 | 0.23482225329274253 |
| Fine-tuned Model | 0.3202412916174494 | 0.1291838902741764 | 0.23471503036165922 | 0.29682001157527327 |

### Interpretation

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a commonly used metric for evaluating the quality of generated summaries by comparing them to reference summaries. In our case, we use it to evaluate the highlights generated by the original base model and the fine-tuned model, using the highlights provided in the dataset as reference.

The ROUGE-1 and ROUGE-2 scores represent unigram and bigram overlap, respectively, while ROUGE-L and ROUGE-Lsum evaluate the longest common subsequence between the generated and reference texts—giving insight into fluency and content preservation.

From the results, it is evident that the fine-tuned model outperforms the original base model across all ROUGE metric indicating a better word-level content capture and phrase-level coherence.

## BertScore

Here, we use the `bert_score` library to calculate the precision, recall, and F1 for the highlights generated by the fine-tuned model and the original base model. We use the highlights in the original dataset as reference.

We followed this [tutorial](https://haticeozbolat17.medium.com/text-summarization-how-to-calculate-bertscore-771a51022964) to write this code.

```python
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from bert_score import BERTScorer

# BERTScore calculation
scorer = BERTScorer(model_type='bert-base-uncased')
P_original_model, R_original_model, F1_original_model = scorer.score(original_model_all_highlights, ground_truth_all_highlights)
P_fine_tuned_model, R_fine_tuned_model, F1_fine_tuned_model = scorer.score(fine_tuned_model_all_highlights, ground_truth_all_highlights)

print(f"BERTScore Precision for original model: {P_original_model.mean():.4f}, Recall: {R_original_model.mean():.4f}, F1: {F1_original_model.mean():.4f}")
print(f"BERTScore Precision for fine-tuned model: {P_fine_tuned_model.mean():.4f}, Recall: {R_fine_tuned_model.mean():.4f}, F1: {F1_fine_tuned_model.mean():.4f}")
```

Console Output:

```
BERTScore Precision for original model: 0.4919, Recall: 0.6337, F1: 0.5528
BERTScore Precision for fine-tuned model: 0.5750, Recall: 0.6091, F1: 0.5904
```

Table View:

| Model | Precision | Recall | F1 |
|-------|-----------|--------|----|
| Original Base Model | 0.4919 | 0.6337 | 0.5528 |
| Fine-tuned Model | 0.5750 | 0.6091 | 0.5904 |

### Interpretation

To further evaluate model performance, we report precision, recall, and F1 scores. These metrics offer complementary views of how well the generated highlights align with the reference highlights.

- Precision measures the proportion of generated highlights that are relevant (i.e., present in the reference).

- Recall measures the proportion of reference highlights that are correctly captured by the model.

- F1 Score is the harmonic mean of Precision and Recall, balancing both aspects.

From the table, we observe that the fine-tuned model achieves higher Precision (0.5750 vs. 0.4919) and F1 score (0.5904 vs. 0.5528), indicating that it generates more relevant and balanced highlights overall. Interestingly, while recall slightly decreases (0.6091 vs. 0.6337), this trade-off is acceptable as the improvement in precision leads to a more meaningful and concise summary generation.

These results reinforce the benefit of fine-tuning, showing that it leads to more accurate and focused highlight generation without significantly sacrificing recall.

## Inference Time Analysis

Next we compare the inference time between the fine-tuned model and the original base model. Inference time is a critical metric for evaluating the practical usability of a model, especially in real-time or resource-constrained environments.

### Basic Stats

```python
# Basic Stats

import numpy as np

def print_inference_stats(times, model_name):
    times = np.array(times)
    print(f"--- {model_name} ---")
    print(f"Mean:    {np.mean(times):.4f} ms")
    print(f"Median:  {np.median(times):.4f} ms")
    print(f"Std Dev: {np.std(times):.4f} ms")
    print(f"Min:     {np.min(times):.4f} ms")
    print(f"Max:     {np.max(times):.4f} ms")
    print()

print_inference_stats(original_model_all_times, "Model A")
print_inference_stats(fine_tuned_model_all_times, "Model B")
```

Console Output (Model A is the original base model and Model B is the fine-tuned model):

```
--- Model A ---
Mean:    11.4647 sec
Median:  11.3639 sec
Std Dev: 1.5827 sec
Min:     8.9727 sec
Max:     16.8651 sec

--- Model B ---
Mean:    7.5664 sec
Median:  7.3395 sec
Std Dev: 2.2447 sec
Min:     3.5611 sec
Max:     17.9274 sec
```

Table View:

| Model | Mean | Median | Std Dev | Min | Max |
|-------|------|--------|---------|-----|-----|
| Original Base Model | 11.4647 sec | 11.3639 sec | 1.5827 sec | 8.9727 sec | 16.8651 sec |
| Fine-tuned Model | 7.5664 sec | 7.3395 sec | 2.2447 sec | 3.5611 sec | 17.9274 sec |

#### Interpretation

From the statistics:

- Original Base Model (Model A) has a mean inference time of 11.46 seconds, with relatively low variability (standard deviation of 1.58 sec). Its inference times are more consistent, with a narrower range between the minimum (8.97 sec) and maximum (16.87 sec).

Fine-tuned Model (Model B) is significantly faster, with a mean inference time of 7.57 seconds, representing a ~34% reduction in average runtime. However, it shows greater variability (standard deviation of 2.24 sec), and a wider range, from as low as 3.56 sec to as high as 17.93 sec.

Overall, while the fine-tuned model is faster on average, its inference time is less stable. Depending on the deployment scenario, this trade-off may be acceptable or even desirable, especially if lower average latency is prioritized over consistency.

### Box Plot

The box plot highlights key summary statistics such as the median, interquartile range, and potential outliers.

```python
import matplotlib.pyplot as plt

# Box Plot (for quick comparison of median, spread, outliers)
plt.boxplot([original_model_all_times, fine_tuned_model_all_times], labels=["Original Model", "Fine-tuned Model"])
plt.ylabel("Inference Time (ms)")  # or whatever unit
plt.title("Comparison of Inference Times")
plt.grid(True)
plt.show()
```

![](Box Plot.png)

#### Interpretation

The box plot compares the distribution of inference times for the original base model and the fine-tuned model.

- The fine-tuned model has a noticeably lower median inference time, indicating faster performance most of the time. However, it also shows greater variability, with a wider spread and more outliers, especially on the higher end.

- In contrast, the original model has a higher and more stable median, with a tighter interquartile range and fewer extreme values.

This visual reinforces the earlier statistical findings: while the fine-tuned model offers faster inference overall, it introduces more variability in runtime, which may be a consideration for real-time or latency-sensitive applications.


### Violin Plot

The violin plot adds a smoothed density estimate, providing a clearer picture of the distribution’s shape and variance.

```python
# Violin Plot (fancier version of boxplot + distribution)

plt.violinplot([original_model_all_times, fine_tuned_model_all_times], showmeans=True)
plt.xticks([1, 2], ["Original Model", "Fine-tuned Model"])
plt.ylabel("Inference Time (ms)")
plt.title("Violin Plot of Inference Times")
plt.grid(True)
plt.show()
```

![](Violin Plot.png)

#### Interpretation

The violin plot provides a detailed view of the distribution shape for inference times of both models.

- The fine-tuned model, while centered at a lower median (~7 seconds), displays a wider and more asymmetric distribution, with some extreme values stretching as high as 18 seconds. The density bulge around 6–8 seconds for the fine-tuned model suggests that most inferences are fast, but occasional spikes in inference time contribute to its higher variability.

- The original base model shows a tight distribution centered around 11–12 seconds, with a smaller spread. This indicates consistent performance with few outliers and low variance.

This visualization reinforces the trade-off: the fine-tuned model is typically faster but less predictable, whereas the original model offers more consistent but slower inference times.


### Cumulative Distribution Function (CDF)

The Cumulative Distribution Function (CDF) plot shows the proportion of inferences completed within a given time, helping visualize how quickly each model reaches different latency thresholds.

```python
# Cumulative Distribution Function (CDF) (good for seeing which model is faster X% of the time)

# Sort the times
original_model_all_times_sorted = np.sort(original_model_all_times)
fine_tuned_model_all_times_sorted = np.sort(fine_tuned_model_all_times)

# Compute cumulative probabilities
cdf_original_times = np.arange(len(original_model_all_times_sorted)) / len(original_model_all_times_sorted)
cdf_fine_tuned_times = np.arange(len(fine_tuned_model_all_times_sorted)) / len(fine_tuned_model_all_times_sorted)

plt.plot(original_model_all_times_sorted, cdf_original_times, label="Original Model")
plt.plot(fine_tuned_model_all_times_sorted, cdf_fine_tuned_times, label="Fine-tuned Model")
plt.xlabel("Inference Time (ms)")
plt.ylabel("Cumulative Probability")
plt.title("CDF of Inference Times")
plt.legend()
plt.grid(True)
plt.show()
```

![](CDF.png)

#### Interpretation

The CDF plot illustrates the proportion of inference runs completed within a given amount of time for both models.

- The fine-tuned model consistently completes inferences faster: over 80% of its inferences finish before 9 seconds, which is approximately the fastest inference time achieved by the original model. This highlights a substantial latency improvement. However, the fine-tuned model also shows a longer tail, with a few outlier inferences taking up to 18 seconds. 

- In contrast, the original model has a more compact range, with all inferences completing between roughly 9 and 17 seconds.

Overall, the fine-tuned model delivers faster results for the majority of inputs, reinforcing its advantage in latency-critical settings.